In [35]:
from kfunction import read_excel_data
import pandas as pd
import numpy as np
import win32com.client as win32
import os
import datetime

In [36]:
excel_file_path = "data\\협력사단품별매입(최종작업용).xlsx"
excel_file_path2 = "data\\기준(최종작업용).xlsx"
excel_file_path3 = "data\\매입세금계산서.xlsx"
excel_file_path4 = "data\\지불보조장.xlsx"
excel_file_path5 = "data\\매입세금계산서(WIS).xlsx"
excel_file_path6 = "data\\임가공비.xlsx"

In [37]:
df_standard= read_excel_data(excel_file_path2, sheet=0)
df_standard.head()


[INFO] 'data\기준(최종작업용).xlsx' 읽는 중…


,협력사코드,협력사명,단품코드,단품명
0,424488,(유)담채원,1845027,"[산들]유기농 김치(김장대행)(냉장,국산)"
1,424488,(유)담채원,1845034,"[산들]김치양념소(택배용)(실온,국산)"
2,425010,(식경)대흥푸드,1832273,"(G)새우튀김(대흥푸드,냉동)"
3,425244,(식)한만두식품,1849963,(수출용) 언리미트 채식교자(냉동)
4,425244,(식)한만두식품,1849964,(수출용) 언리미트 김치채식교자(냉동)


In [38]:
df = read_excel_data(excel_file_path, header=0)
# Grand Total 행(이제 0번 인덱스) 제거
df = df.drop(0).reset_index(drop=True)
df.head()

[INFO] 'data\협력사단품별매입(최종작업용).xlsx' 읽는 중…


,년월,협력사코드,협력사명,단품코드,단품명,면과세구분명,매입확정수량,매입에누리금액,매입장려금금액,매입조정금액,매입금액,최종매입금액
0,202401.0,100000.0,(식경)미존재협력사,1242181.0,"톳나물(상품,국산)",면세,0.0,0.0,0.0,0.0,0.0,0.0
1,202401.0,100000.0,(식경)미존재협력사,1242277.0,"적채(상품,냉장,국산)",면세,0.0,0.0,0.0,0.0,0.0,0.0
2,202401.0,100000.0,(식경)미존재협력사,1242466.0,"애플민트(상품,실온,국산)",면세,0.0,0.0,0.0,0.0,0.0,0.0
3,202401.0,100000.0,(식경)미존재협력사,1246736.0,"오렌지(상품,수입)",면세,0.0,0.0,0.0,0.0,0.0,0.0
4,202401.0,100000.0,(식경)미존재협력사,1251319.0,"물미역(냉장,상품,국산)",면세,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
# 각 그룹별로 최종매입금액은 합계, 협력사명과 단품명은 첫번째 값을 취하도록 그룹화합니다.
# 년월, 협력사코드, 단품코드, 면과세구분명을 기준으로 그룹화하여
# 최종매입금액은 합계, 협력사명과 단품명은 첫번째 값을 취하도록 집계합니다.
df_pivot = df.groupby(["년월", "협력사코드", "단품코드", "면과세구분명"]).agg({
    "최종매입금액": "sum",
    "협력사명": "first",
    "단품명": "first"
}).reset_index()

# 원본 DataFrame의 컬럼 순서(년월, 협력사코드, 협력사명, 단품코드, 단품명, 면과세구분명, 최종매입금액)를 최대한 유지합니다.
df_pivot = df_pivot[["년월", "협력사코드", "협력사명", "단품코드", "단품명", "면과세구분명", "최종매입금액"]]

# 결과 확인
df_pivot.head()


,년월,협력사코드,협력사명,단품코드,단품명,면과세구분명,최종매입금액
0,202401.0,100000.0,(식경)미존재협력사,1242181.0,"톳나물(상품,국산)",면세,0.0
1,202401.0,100000.0,(식경)미존재협력사,1242277.0,"적채(상품,냉장,국산)",면세,0.0
2,202401.0,100000.0,(식경)미존재협력사,1242466.0,"애플민트(상품,실온,국산)",면세,0.0
3,202401.0,100000.0,(식경)미존재협력사,1246736.0,"오렌지(상품,수입)",면세,0.0
4,202401.0,100000.0,(식경)미존재협력사,1251319.0,"물미역(냉장,상품,국산)",면세,0.0


In [40]:
# df_standard에서 필요한 열만 선택하고 중복 제거 (협력사코드와 단품코드 조합 기준)
df_standard_subset = df_standard[['협력사코드', '단품코드']].drop_duplicates(subset=['협력사코드', '단품코드'])

df_pivot['협력사코드'] = df_pivot['협력사코드'].astype(int).astype(str)
df_standard_subset['협력사코드'] = df_standard_subset['협력사코드'].astype(int).astype(str)

# 3. 단품코드도 동일하게 처리
df_pivot['단품코드'] = df_pivot['단품코드'].astype(int).astype(str)
df_standard_subset['단품코드'] = df_standard_subset['단품코드'].astype(int).astype(str)

# df_pivot과 df_standard_subset을 inner join (월별, 협력사코드, 단품코드 각각의 조합은 pivot 단계에서 유일해야 함)
df_final = pd.merge(df_pivot, df_standard_subset, on=['협력사코드', '단품코드'], how='inner')

# 결과 확인
df_final.head()

,년월,협력사코드,협력사명,단품코드,단품명,면과세구분명,최종매입금액
0,202401.0,427211,(식)지원(OMS),1528921,[용기](G)용기)JB-123 뚜껑(실온),과세,738000.0
1,202401.0,427211,(식)지원(OMS),1528922,[용기](G)용기)JB-033 뚜껑(실온),과세,3214800.0
2,202401.0,427211,(식)지원(OMS),1538376,(G)햄버거 깍지_소_종이(실온),과세,626400.0
3,202401.0,427211,(식)지원(OMS),1560708,[용기][G]샌드위치 용기(HGF-21),과세,3348000.0
4,202401.0,427717,(식경)GM글로벌(OMS),1479650,(G)열성형포장필름(CK상단) 5kg용(실온),과세,1554800.0


In [41]:
# df_final의 협력사코드 기준으로 그룹화하여, 협력사명은 첫번째 값을 사용하고 최종매입금액은 합계를 구합니다.
# df_final의 년월과 협력사코드를 기준으로 그룹화하여, 협력사명은 첫번째 값을 사용하고 최종매입금액은 합계를 구합니다.
df_final_pivot = df_final.groupby(["년월", "협력사코드","면과세구분명"]).agg({
    "협력사명": "first",
    "최종매입금액": "sum"
}).reset_index()

# 원본 DataFrame의 컬럼 순서(년월, 협력사코드, 협력사명, 최종매입금액)를 최대한 유지합니다.
df_final_pivot = df_final_pivot[["년월", "협력사코드", "협력사명",  "면과세구분명", "최종매입금액"]]

# 결과 확인
df_final_pivot.head()

,년월,협력사코드,협력사명,면과세구분명,최종매입금액
0,202401.0,427211,(식)지원(OMS),과세,7927200.0
1,202401.0,427717,(식경)GM글로벌(OMS),과세,1554800.0
2,202401.0,427720,(식경)목양산업(OMS),과세,499500.0
3,202401.0,428262,(식경)한국이노팩(OMS),과세,12930000.0
4,202401.0,429710,(식)세림비앤지-신(OMS),과세,27312480.0


In [42]:
# 협력사코드, 년월 순으로 정렬 (오름차순)
df_final_pivot = df_final_pivot.sort_values(by=["협력사코드", "년월","면과세구분명"])
df_final_pivot["key"] = df_final_pivot["년월"].astype(int).astype(str) + df_final_pivot["협력사코드"].astype(str) + df_final_pivot["면과세구분명"]
df_final_pivot.head()
df_final_pivot = df_final_pivot[df_final_pivot.최종매입금액!= 0]

In [43]:
df_final_pivot

,년월,협력사코드,협력사명,면과세구분명,최종매입금액,key
0,202401.0,427211,(식)지원(OMS),과세,7927200.0,202401427211과세
6,202402.0,427211,(식)지원(OMS),과세,10426800.0,202402427211과세
11,202403.0,427211,(식)지원(OMS),과세,9591600.0,202403427211과세
18,202404.0,427211,(식)지원(OMS),과세,8137800.0,202404427211과세
26,202405.0,427211,(식)지원(OMS),과세,8224800.0,202405427211과세
1,202401.0,427717,(식경)GM글로벌(OMS),과세,1554800.0,202401427717과세
7,202402.0,427717,(식경)GM글로벌(OMS),과세,8064680.0,202402427717과세
12,202403.0,427717,(식경)GM글로벌(OMS),과세,8051160.0,202403427717과세
19,202404.0,427717,(식경)GM글로벌(OMS),과세,8057920.0,202404427717과세
27,202405.0,427717,(식경)GM글로벌(OMS),과세,8037640.0,202405427717과세


In [44]:
df_num = read_excel_data(excel_file_path5)
# 결과 확인
print(df_num.head())

[INFO] 'data\매입세금계산서(WIS).xlsx' 읽는 중…
     센터 협력사승인여부    계산서구분      계산서작성일   협력사코드        협력사명           협력사E-MAIL  \
0  영남센터      승인    일반계산서  2024-01-02  423191  (원)한라에스앤에프                None   
1  영남센터      승인    일반계산서  2024-01-03  426603     (원)아름드리  ard*****@naver.com   
2  경인센터      승인  일반세금계산서  2024-01-03  430472      (원)꼬미다      bil*@comida.kr   
3  경인센터      승인  일반세금계산서  2024-01-03  429988      (원)아우노        bil*@auno.kr   
4  영남센터      승인    일반계산서  2024-01-04  430721      (원)미트웍  mea*****@naver.com   

  발행구분 접수/발행 계산서상태  ...           총액                  매입번호    담당자사번  담당자명  \
0  역발행    접수    정상  ...  354060000.0  BB202401040000039294  1250161   박철중   
1  역발행    접수    정상  ...   45450000.0  BB202401030000039155  1203533   이*용   
2  역발행    접수    정상  ...   32848200.0  BB202401030000039157  1100784   임*곤   
3  역발행    접수    정상  ...   16544352.0  BB202401030000039156  1100784   임*곤   
4  역발행    접수    정상  ...   14351400.0  BB202401040000039314  1405745   이*원   

           HIVS승인번

In [45]:
df_tax = df_num[["협력사코드", "계산서작성일", "협력사명", "계산서구분", "사업자번호", "공급가액", "세액", "국세청승인번호"]]

df_tax.head()

,협력사코드,계산서작성일,협력사명,계산서구분,사업자번호,공급가액,세액,국세청승인번호
0,423191,2024-01-02,(원)한라에스앤에프,일반계산서,616-81-47134,354060000.0,0.0,202401021024010273554432.0
1,426603,2024-01-03,(원)아름드리,일반계산서,606-38-36707,45450000.0,0.0,202401031024010311237632.0
2,430472,2024-01-03,(원)꼬미다,일반세금계산서,546-37-01156,29862000.0,2986200.0,202401031024010311237632.0
3,429988,2024-01-03,(원)아우노,일반세금계산서,502-86-27533,15040320.0,1504032.0,2024010341000026zzzhax74
4,430721,2024-01-04,(원)미트웍,일반계산서,130-86-76770,14351400.0,0.0,2024010441000203000007d5


In [46]:
df_final_pivot['협력사코드'] = df_final_pivot['협력사코드'].astype(str)
df_tax['협력사코드'] = df_tax['협력사코드'].astype(str)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_34320\2987439431.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tax['협력사코드'] = df_tax['협력사코드'].astype(str)


In [47]:
df_tax_sort = df_tax[df_tax.협력사코드.isin(df_final_pivot.협력사코드.tolist())]
df_tax_sort.head()

,협력사코드,계산서작성일,협력사명,계산서구분,사업자번호,공급가액,세액,국세청승인번호
750,429710,2024-01-31,(식)세림비앤지-신(OMS),일반세금계산서,262-87-01457,156000.0,15600.0,2024013142000076y0258221
755,427211,2024-01-31,(식)지원(OMS),일반세금계산서,128-81-24750,1398915.0,139892.0,2024013142000076y0258306
837,427211,2024-01-31,(식)지원(OMS),일반세금계산서,128-81-24750,121437.0,12144.0,2024013142000076y0258307
1409,429710,2024-01-31,(식)세림비앤지-신(OMS),일반세금계산서,262-87-01457,5399280.0,539928.0,2024013142000076y0255900
1412,428262,2024-01-31,(식경)한국이노팩(OMS),일반세금계산서,387-81-00191,57037360.0,5703736.0,2024013142000076y0256132


In [48]:
# df_tax_hifi 읽기 (MultiIndex 헤더)
df_tax_hifi = read_excel_data(excel_file_path3, sheet=0, header=[0,1])

# MultiIndex 컬럼 평탄화
df_tax_hifi.columns = [col[0] if pd.isna(col[1]) else f"{col[0]}_{col[1]}" for col in df_tax_hifi.columns]

# 컬럼 매핑 정의
column_mapping = {
    '국세청승인번호': '국세청승인번호',
    '업체사업자번호': '업체사업자번호', 
    '국세청작성일': 'nan_작성일',
    '국세청발급일': 'nan_발급일'
}

# lookup_df 생성
lookup_df = df_tax_hifi[[column_mapping['국세청승인번호'], 
                        column_mapping['업체사업자번호'], 
                        column_mapping['국세청작성일'],
                        column_mapping['국세청발급일']]].drop_duplicates(subset=column_mapping['국세청승인번호'], keep='first')

# 컬럼명을 원래 이름으로 변경
lookup_df = lookup_df.rename(columns={
    column_mapping['국세청승인번호']: '국세청승인번호',
    column_mapping['업체사업자번호']: '업체사업자번호',
    column_mapping['국세청작성일']: '국세청작성일',
    column_mapping['국세청발급일']: '국세청발급일'
})

# df_tax_sort와 lookup_df를 '국세청승인번호'를 기준으로 병합
df_tax_new = pd.merge(df_tax_sort, lookup_df, on='국세청승인번호', how='left')

# 병합된 '국세청작성일', '국세청발급일' 컬럼을 datetime형으로 변환
df_tax_new['국세청작성일'] = pd.to_datetime(df_tax_new['국세청작성일'], errors='coerce')
df_tax_new['국세청발급일'] = pd.to_datetime(df_tax_new['국세청발급일'], errors='coerce')

# 만약 tz-aware datetime이라면 timezone 제거
if pd.api.types.is_datetime64tz_dtype(df_tax_new['국세청작성일']):
    df_tax_new['국세청작성일'] = df_tax_new['국세청작성일'].dt.tz_convert(None)
    
if pd.api.types.is_datetime64tz_dtype(df_tax_new['국세청발급일']):
    df_tax_new['국세청발급일'] = df_tax_new['국세청발급일'].dt.tz_convert(None)
    
# 업체사업자번호 정리
df_tax_new["업체사업자번호"] = df_tax_new["업체사업자번호"].astype(str).str.replace("-", "", regex=True)

[INFO] 'data\매입세금계산서.xlsx' 읽는 중…


c:\Users\Administrator\Desktop\subcontract\kfunction.py:54: PerformanceWarning: indexing past lexsort depth may impact performance.
  column_data = df[col]
C:\Users\Administrator\AppData\Local\Temp\ipykernel_34320\4237159728.py:37: DeprecationWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if pd.api.types.is_datetime64tz_dtype(df_tax_new['국세청작성일']):
C:\Users\Administrator\AppData\Local\Temp\ipykernel_34320\4237159728.py:40: DeprecationWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if pd.api.types.is_datetime64tz_dtype(df_tax_new['국세청발급일']):


In [49]:
df_tax_new.sort_values(by='국세청발급일', ascending=True)

,협력사코드,계산서작성일,협력사명,계산서구분,사업자번호,공급가액,세액,국세청승인번호,업체사업자번호,국세청작성일,국세청발급일
5,427717,2024-01-31,(식경)GM글로벌(OMS),일반세금계산서,405-03-51248,10590800.0,1059080.0,2024013142000076y0254562,4050351248,2024-01-31,2024-02-02
7,427720,2024-01-31,(식경)목양산업(OMS),일반세금계산서,129-86-41169,43567260.0,4356726.0,2024013142000076y0254801,1298641169,2024-01-31,2024-02-02
3,429710,2024-01-31,(식)세림비앤지-신(OMS),일반세금계산서,262-87-01457,5399280.0,539928.0,2024013142000076y0255900,2628701457,2024-01-31,2024-02-05
4,428262,2024-01-31,(식경)한국이노팩(OMS),일반세금계산서,387-81-00191,57037360.0,5703736.0,2024013142000076y0256132,3878100191,2024-01-31,2024-02-05
6,427211,2024-01-31,(식)지원(OMS),일반세금계산서,128-81-24750,43033900.0,4303390.0,2024013142000076y0257688,1288124750,2024-01-31,2024-02-05
...,...,...,...,...,...,...,...,...,...,...,...
79,430524,2024-05-31,(식수)와이케이원,일반세금계산서,302-86-01438,9408000.0,940800.0,2024053142000076y0207440,3028601438,2024-05-31,2024-06-05
80,429710,2024-05-31,(식)세림비앤지-신(OMS),일반세금계산서,262-87-01457,468000.0,46800.0,2024053142000076y0207474,2628701457,2024-05-31,2024-06-05
69,430448,2024-05-31,(식수)동보식품-신,일반세금계산서,652-81-02787,249755820.0,24975582.0,2024053142000076y0208092,6528102787,2024-05-31,2024-06-05
82,429710,2024-05-31,(식)세림비앤지-신(OMS),일반세금계산서,262-87-01457,78000.0,7800.0,2024053142000076y0207200,2628701457,2024-05-31,2024-06-05


In [50]:
import pandas as pd
import numpy as np

# =====================================
# 전처리: df_tax_new 데이터셋 준비
# =====================================
# Step 1: df_tax_new의 '국세청작성일' 컬럼을 datetime형으로 변환 (에러 발생 시 NaT로 처리)
df_tax_new['국세청작성일'] = pd.to_datetime(df_tax_new['국세청작성일'], errors='coerce')
if pd.api.types.is_datetime64tz_dtype(df_tax_new['국세청작성일']):
    df_tax_new['국세청작성일'] = df_tax_new['국세청작성일'].dt.tz_convert(None)

# 작성년도와 작성월 추출
df_tax_new['작성년도'] = df_tax_new['국세청작성일'].dt.year
df_tax_new['작성월'] = df_tax_new['국세청작성일'].dt.month

# 공급가액과 세액을 숫자형으로 변환 (문제가 있는 값은 NaN 처리)
df_tax_new["공급가액"] = pd.to_numeric(df_tax_new["공급가액"], errors="coerce")
df_tax_new["세액"] = pd.to_numeric(df_tax_new["세액"], errors="coerce")

# df_tax_new에 '대사여부' 및 '구분키' 컬럼 추가 (초기값은 빈 문자열)
df_tax_new['대사여부'] = ""
df_tax_new['구분키'] = ""

# =====================================
# 전처리: df_final_pivot 데이터셋 준비
# =====================================
# Step 2: df_final_pivot의 '년월' 컬럼에서 작성년도와 월 추출 (예: '202312' → 2023, 12)
df_final_pivot['년'] = df_final_pivot['년월'].astype(str).str[:4].astype(int)
df_final_pivot['월'] = df_final_pivot['년월'].astype(str).str[4:6].astype(int)

# Step 3: df_final_pivot에 국세청 관련 데이터를 저장할 새 컬럼 및 구분키 추가
df_final_pivot['국세청작성일'] = None
df_final_pivot['국세청발급일'] = None      # 국세청발급일 컬럼 추가
df_final_pivot['국세청공급가액'] = None
df_final_pivot['국세청세액'] = None
df_final_pivot['구분키'] = ""
# 추가: 국세청승인번호와 업체사업자번호 컬럼 추가
df_final_pivot['국세청승인번호'] = None
df_final_pivot['업체사업자번호'] = None

# float 비교를 위한 허용 오차
tolerance = 1e-6


# =====================================
# Step A: 금액대사 (1:1 대사) 처리
# =====================================
for idx, row in df_final_pivot.iterrows():
    협력사코드_final = row['협력사코드']
    년도_final = row['년']
    월_final = row['월']
    금액_final = row['최종매입금액']

    # "과세" 또는 "영세"인 경우 → "일반세금계산서",
    # "면세"인 경우 → "일반계산서"
    if row['면과세구분명'] in ["과세", "영세"]:
        invoice_condition = "일반세금계산서"
    else:
        invoice_condition = "일반계산서"

    condition = (
        (df_tax_new['협력사코드'] == 협력사코드_final) &
        (df_tax_new['작성년도'] == 년도_final) &
        (df_tax_new['작성월'] == 월_final) &
        (df_tax_new['공급가액'] == 금액_final) &
        (df_tax_new['대사여부'] == "") &
        (df_tax_new['계산서구분'] == invoice_condition)
    )
    df_candidates = df_tax_new.loc[condition]

    if not df_candidates.empty:
        first_index = df_candidates.index[0]
        mapped_date = df_tax_new.at[first_index, '국세청작성일']
        mapped_issue_date = df_tax_new.at[first_index, '국세청발급일']
        mapped_supply = df_tax_new.at[first_index, '공급가액']
        mapped_tax = df_tax_new.at[first_index, '세액']

        df_final_pivot.at[idx, '국세청작성일'] = mapped_date
        df_final_pivot.at[idx, '국세청발급일'] = mapped_issue_date
        df_final_pivot.at[idx, '국세청공급가액'] = mapped_supply
        df_final_pivot.at[idx, '국세청세액'] = mapped_tax
        df_final_pivot.at[idx, '구분키'] = "금액대사"
        df_final_pivot.at[idx, '국세청승인번호'] = df_tax_new.at[first_index, '국세청승인번호']
        df_final_pivot.at[idx, '업체사업자번호'] = df_tax_new.at[first_index, '업체사업자번호']

        # 1:1 매칭이므로 번호는 -1로 표시
        df_tax_new.at[first_index, '대사여부'] = f"{row['key']}-1"
        df_tax_new.at[first_index, '구분키'] = "금액대사"


# =====================================
# Step A-2: 금액대사(수기확인) 처리
# (면과세구분명에 따른 계산서구분 조건을 제외하고 1:1 매칭)
# =====================================
for idx, row in df_final_pivot.iterrows():
    # 이미 매핑된 경우 건너뛰기
    if pd.notnull(row['국세청작성일']):
        continue

    협력사코드_final = row['협력사코드']
    년도_final = row['년']
    월_final = row['월']
    금액_final = row['최종매입금액']

    # 계산서구분 조건(면과세 구분에 따른 조건)을 제외한 조건으로 매칭
    condition = (
        (df_tax_new['협력사코드'] == 협력사코드_final) &
        (df_tax_new['작성년도'] == 년도_final) &
        (df_tax_new['작성월'] == 월_final) &
        (df_tax_new['공급가액'] == 금액_final) &
        (df_tax_new['대사여부'] == "")
    )
    df_candidates = df_tax_new.loc[condition]

    if not df_candidates.empty:
        first_index = df_candidates.index[0]
        mapped_date = df_tax_new.at[first_index, '국세청작성일']
        mapped_issue_date = df_tax_new.at[first_index, '국세청발급일']
        mapped_supply = df_tax_new.at[first_index, '공급가액']
        mapped_tax = df_tax_new.at[first_index, '세액']

        df_final_pivot.at[idx, '국세청작성일'] = mapped_date
        df_final_pivot.at[idx, '국세청발급일'] = mapped_issue_date
        df_final_pivot.at[idx, '국세청공급가액'] = mapped_supply
        df_final_pivot.at[idx, '국세청세액'] = mapped_tax
        df_final_pivot.at[idx, '구분키'] = "금액대사(수기확인)"
        df_final_pivot.at[idx, '국세청승인번호'] = df_tax_new.at[first_index, '국세청승인번호']
        df_final_pivot.at[idx, '업체사업자번호'] = df_tax_new.at[first_index, '업체사업자번호']

        df_tax_new.at[first_index, '대사여부'] = f"{row['key']}-1"
        df_tax_new.at[first_index, '구분키'] = "금액대사(수기확인)"


# =====================================
# Step B: 순차대사 (1:N 대사) 처리
# (금액대사 또는 금액대사(수기확인)로 매핑되지 않은 pivot 행에 대해 처리)
# =====================================
for idx, row in df_final_pivot.iterrows():
    # 이미 매핑 완료된 pivot은 건너뜁니다.
    if pd.notnull(row['국세청작성일']):
        continue

    협력사코드_final = row['협력사코드']
    년도_final = row['년']
    월_final = row['월']
    target_amount = row['최종매입금액']

    if row['면과세구분명'] in ["과세", "영세"]:
        invoice_condition = "일반세금계산서"
    else:
        invoice_condition = "일반계산서"

    candidates = df_tax_new[
        (df_tax_new['협력사코드'] == 협력사코드_final) &
        (df_tax_new['작성년도'] == 년도_final) &
        (df_tax_new['작성월'] == 월_final) &
        (df_tax_new['대사여부'] == "") &
        (df_tax_new['계산서구분'] == invoice_condition)
    ]

    if candidates.empty:
        continue

    # 후보들을 국세청작성일 기준 오름차순(FIFO)로 정렬
    candidates = candidates.sort_values(by='국세청작성일')
    cumulative_sum = 0.0
    selected_indices = []

    for cand_idx, cand_row in candidates.iterrows():
        cumulative_sum += cand_row['공급가액']
        selected_indices.append(cand_idx)

        if np.abs(cumulative_sum - target_amount) < tolerance:
            mapped_date = candidates.loc[selected_indices, '국세청작성일'].min()
            mapped_issue_date = candidates.loc[selected_indices, '국세청발급일'].min()
            mapped_supply = cumulative_sum
            mapped_tax = candidates.loc[selected_indices, '세액'].sum()

            df_final_pivot.at[idx, '국세청작성일'] = mapped_date
            df_final_pivot.at[idx, '국세청발급일'] = mapped_issue_date
            df_final_pivot.at[idx, '국세청공급가액'] = mapped_supply
            df_final_pivot.at[idx, '국세청세액'] = mapped_tax
            df_final_pivot.at[idx, '구분키'] = "순차대사"
            # 여러 건 중 첫번째 후보의 국세청승인번호와 업체사업자번호 할당
            first_candidate_index = selected_indices[0]
            df_final_pivot.at[idx, '국세청승인번호'] = candidates.loc[first_candidate_index, '국세청승인번호']
            df_final_pivot.at[idx, '업체사업자번호'] = candidates.loc[first_candidate_index, '업체사업자번호']

            # 각 후보에 대해 기본 key에 순번을 붙여 업데이트 (예: xxx-1, xxx-2)
            for i, si in enumerate(selected_indices, start=1):
                df_tax_new.at[si, '대사여부'] = f"{row['key']}-{i}"
                df_tax_new.at[si, '구분키'] = "순차대사"
            break

        elif cumulative_sum > target_amount:
            selected_indices = []
            break


# =====================================
# Step C: 부분대사 처리 (1:1 매칭, df_tax_new의 공급가액이 pivot의 최종매입금액보다 큰 경우)
# =====================================
for idx, row in df_final_pivot.iterrows():
    if pd.notnull(row['국세청작성일']):
        continue

    협력사코드_final = row['협력사코드']
    년도_final = row['년']
    월_final = row['월']
    target_amount = row['최종매입금액']

    if row['면과세구분명'] in ["과세", "영세"]:
        invoice_condition = "일반세금계산서"
    else:
        invoice_condition = "일반계산서"

    candidates = df_tax_new[
        (df_tax_new['협력사코드'] == 협력사코드_final) &
        (df_tax_new['작성년도'] == 년도_final) &
        (df_tax_new['작성월'] == 월_final) &
        (df_tax_new['대사여부'] == "") &
        (df_tax_new['계산서구분'] == invoice_condition)
    ]

    if candidates.empty:
        continue

    # 공급가액이 반드시 target_amount보다 큰 후보만 선택
    candidates_gt = candidates[candidates['공급가액'] > target_amount]
    if candidates_gt.empty:
        continue  # 조건을 만족하는 후보가 없으면 매핑하지 않음

    # 차이가 최소인 항목 선택 (1:1 매칭)
    candidate_index = (candidates_gt['공급가액'] - target_amount).abs().idxmin()
    candidate_row = candidates_gt.loc[candidate_index]

    mapped_date = candidate_row['국세청작성일']
    mapped_issue_date = candidate_row['국세청발급일']
    mapped_supply = candidate_row['공급가액']
    mapped_tax = candidate_row['세액']

    df_final_pivot.at[idx, '국세청작성일'] = mapped_date
    df_final_pivot.at[idx, '국세청발급일'] = mapped_issue_date
    df_final_pivot.at[idx, '국세청공급가액'] = mapped_supply
    df_final_pivot.at[idx, '국세청세액'] = mapped_tax
    df_final_pivot.at[idx, '구분키'] = "부분대사"
    df_final_pivot.at[idx, '국세청승인번호'] = candidate_row['국세청승인번호']
    df_final_pivot.at[idx, '업체사업자번호'] = candidate_row['업체사업자번호']

    # 1:1 매칭이므로 번호는 -1로 표시
    df_tax_new.at[candidate_index, '대사여부'] = f"{row['key']}-1"
    df_tax_new.at[candidate_index, '구분키'] = "부분대사"


# =====================================
# Step D: 부분대사(수기확인) 처리 (여러 후보 합산 후 매칭)
# (후보들의 합산 금액이 target_amount보다 크면, 국세청작성일과 국세청발급일 중 늦은 날짜로 매핑)
# =====================================
for idx, row in df_final_pivot.iterrows():
    if pd.notnull(row['국세청작성일']):
        continue

    협력사코드_final = row['협력사코드']
    년도_final = row['년']
    월_final = row['월']
    target_amount = row['최종매입금액']

    if row['면과세구분명'] in ["과세", "영세"]:
        invoice_condition = "일반세금계산서"
    else:
        invoice_condition = "일반계산서"

    candidates = df_tax_new[
        (df_tax_new['협력사코드'] == 협력사코드_final) &
        (df_tax_new['작성년도'] == 년도_final) &
        (df_tax_new['작성월'] == 월_final) &
        (df_tax_new['대사여부'] == "") &
        (df_tax_new['계산서구분'] == invoice_condition)
    ]
    candidates = candidates[candidates['공급가액'] <= target_amount]

    if candidates.empty:
        continue

    # 국세청발급일이 늦은 순으로 정렬 (내림차순)
    candidates = candidates.sort_values(by='국세청발급일', ascending=True)

    cumulative_sum = 0.0
    selected_indices = []

    for cand_idx, cand_row in candidates.iterrows():
        cumulative_sum += cand_row['공급가액']
        selected_indices.append(cand_idx)
        if cumulative_sum > target_amount:
            break

    if cumulative_sum > target_amount:
        representative_date = candidates.loc[selected_indices, '국세청작성일'].max()
        representative_issue_date = candidates.loc[selected_indices, '국세청발급일'].max()
        mapped_supply = cumulative_sum
        mapped_tax = candidates.loc[selected_indices, '세액'].sum()

        df_final_pivot.at[idx, '국세청작성일'] = representative_date
        df_final_pivot.at[idx, '국세청발급일'] = representative_issue_date
        df_final_pivot.at[idx, '국세청공급가액'] = mapped_supply
        df_final_pivot.at[idx, '국세청세액'] = mapped_tax
        df_final_pivot.at[idx, '구분키'] = "수기확인"
        df_final_pivot.at[idx, '국세청승인번호'] = candidates.loc[selected_indices[0], '국세청승인번호']
        df_final_pivot.at[idx, '업체사업자번호'] = candidates.loc[selected_indices[0], '업체사업자번호']

        for i, si in enumerate(selected_indices, start=1):
            df_tax_new.at[si, '대사여부'] = f"{row['key']}-{i}"
            df_tax_new.at[si, '구분키'] = "수기확인"


# =====================================
# Step E: 날짜 포매팅 (df_final_pivot의 '국세청작성일' 및 '국세청발급일' 컬럼: YYYY-MM-DD)
# =====================================
df_final_pivot['국세청작성일'] = pd.to_datetime(df_final_pivot['국세청작성일'], errors='coerce')
if pd.api.types.is_datetime64tz_dtype(df_final_pivot['국세청작성일']):
    df_final_pivot['국세청작성일'] = df_final_pivot['국세청작성일'].dt.tz_convert(None)
df_final_pivot['국세청작성일'] = df_final_pivot['국세청작성일'].dt.strftime('%Y-%m-%d')

df_final_pivot['국세청발급일'] = pd.to_datetime(df_final_pivot['국세청발급일'], errors='coerce')
if pd.api.types.is_datetime64tz_dtype(df_final_pivot['국세청발급일']):
    df_final_pivot['국세청발급일'] = df_final_pivot['국세청발급일'].dt.tz_convert(None)
df_final_pivot['국세청발급일'] = df_final_pivot['국세청발급일'].dt.strftime('%Y-%m-%d')

# =====================================
# 최종 결과 확인
# =====================================
print(df_tax_new.head())
print(df_final_pivot.head())

C:\Users\Administrator\AppData\Local\Temp\ipykernel_34320\214086799.py:9: DeprecationWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if pd.api.types.is_datetime64tz_dtype(df_tax_new['국세청작성일']):
C:\Users\Administrator\AppData\Local\Temp\ipykernel_34320\214086799.py:320: DeprecationWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if pd.api.types.is_datetime64tz_dtype(df_final_pivot['국세청작성일']):


    협력사코드      계산서작성일             협력사명    계산서구분         사업자번호        공급가액  \
0  429710  2024-01-31  (식)세림비앤지-신(OMS)  일반세금계산서  262-87-01457    156000.0   
1  427211  2024-01-31       (식)지원(OMS)  일반세금계산서  128-81-24750   1398915.0   
2  427211  2024-01-31       (식)지원(OMS)  일반세금계산서  128-81-24750    121437.0   
3  429710  2024-01-31  (식)세림비앤지-신(OMS)  일반세금계산서  262-87-01457   5399280.0   
4  428262  2024-01-31   (식경)한국이노팩(OMS)  일반세금계산서  387-81-00191  57037360.0   

          세액                   국세청승인번호     업체사업자번호     국세청작성일     국세청발급일  \
0    15600.0  2024013142000076y0258221  2628701457 2024-01-31 2024-02-06   
1   139892.0  2024013142000076y0258306  1288124750 2024-01-31 2024-02-06   
2    12144.0  2024013142000076y0258307  1288124750 2024-01-31 2024-02-06   
3   539928.0  2024013142000076y0255900  2628701457 2024-01-31 2024-02-05   
4  5703736.0  2024013142000076y0256132  3878100191 2024-01-31 2024-02-05   

   작성년도  작성월              대사여부   구분키  
0  2024    1  202401429710과세-2  수기확인  
1 

C:\Users\Administrator\AppData\Local\Temp\ipykernel_34320\214086799.py:325: DeprecationWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if pd.api.types.is_datetime64tz_dtype(df_final_pivot['국세청발급일']):


In [51]:
df_book= read_excel_data(excel_file_path4)
df_book.head()

[INFO] 'data\지불보조장.xlsx' 읽는 중…


,계정코드,계정과목명,회계일,전표유형,회계단위,전표번호,확정번호,전표상태,증빙순번,라인번호,...,투자,원천반제전표번호,원천반제라인번호,원천배부전표번호,원천배부라인번호,작성자,작성부서,작성일,최종승인자,승인일
0,21012300,외상매입금-FBS,2024-01-01,수기전표,현대그린푸드 본사,2024010100024,350,승인,1,9,...,None,None,None,None,None,김성웅,재무회계파트,2024-01-16,강대림,2024-01-17 11:09:31
1,21012300,외상매입금-FBS,2024-01-02,지급전표분개,현대그린푸드 본사,2024010200020,4,승인,1,1,...,None,2023121500079.0,559.0,None,None,이지은,재무회계파트,2024-01-02,강대림,2024-01-02 15:11:12
2,21012300,외상매입금-FBS,2024-01-02,수기전표,현대그린푸드 본사,2024010200022,12,승인,1,2,...,None,None,None,None,None,김나현,영남지원파트,2024-01-02,강대림,2024-01-02 17:32:40
3,21012300,외상매입금-FBS,2024-01-05,지급전표분개,현대그린푸드 본사,2024010500001,50,승인,1,39,...,None,2024010200022.0,2.0,None,None,이지은,재무회계파트,2024-01-05,강대림,2024-01-05 10:38:55
4,21012300,외상매입금-FBS,2024-01-15,지급전표분개,현대그린푸드 본사,2024011500015,261,승인,1,1,...,None,2023122900356.0,3.0,None,None,이지은,재무회계파트,2024-01-15,강대림,2024-01-15 13:26:39


In [52]:
def convert_vendor_to_string(val):
    if pd.isna(val):
        return ''
    elif isinstance(val, (int, float)):
        # 숫자인 경우 정수로 변환 후 문자열로 (소수점 제거)
        return str(int(val))
    else:
        return str(val)

In [53]:
df_book['거래처번호'] = df_book['거래처번호'].apply(convert_vendor_to_string)

In [54]:
# df_final_pivotd에 있는 업체사업자번호 리스트를 이용하여,
# df_book의 거래처번호 중 일치하는 행만 필터링합니다.
filtered_df_book = df_book[df_book['거래처번호'].isin(df_final_pivot['업체사업자번호'])]
filtered_df_book = filtered_df_book[["계정코드","계정과목명", "회계일", "전표번호", "거래처번호", "거래처명", "차변금액", "대변금액"]]
# 각 셀의 문자열에서 쉼표 제거 후 float 타입으로 변환
filtered_df_book["차변금액"] = filtered_df_book["차변금액"].str.replace(",", "", regex=True).astype(float)
filtered_df_book["대변금액"] = filtered_df_book["대변금액"].str.replace(",", "", regex=True).astype(float)
filtered_df_book = filtered_df_book[filtered_df_book['차변금액'] != 0]
print(filtered_df_book.head())

          계정코드      계정과목명         회계일           전표번호       거래처번호  \
66    21012300  외상매입금-FBS  2024-01-15  2024011500015  6528102787   
311   21012300  외상매입금-FBS  2024-01-15  2024011500015  3028601438   
1404  21012300  외상매입금-FBS  2024-01-15  2024011500015  6528102787   
1406  21012300  외상매입금-FBS  2024-01-15  2024011500015  1288124750   
4774  21012300  외상매입금-FBS  2024-01-31  2024013105247  3028601438   

                    거래처명         차변금액       대변금액  
66             동보식품 주식회사  229110200.0        NaN  
311   주식회사 와이케이원(YK.one)   10841600.0        NaN  
1404           동보식품 주식회사     594660.0        NaN  
1406               (주)지원    4264965.0        NaN  
4774  주식회사 와이케이원(YK.one)          NaN  5420800.0  


In [55]:
filtered_df_book.shape

(301, 8)

In [56]:
df_final_pivot["지불예상금액"] = df_final_pivot["국세청공급가액"] + df_final_pivot["국세청세액"]

In [57]:
import pandas as pd
import numpy as np

def find_subset_sum_all_combinations(candidates, target_amount, tolerance=1e-6):
    """
    candidates: 차변금액이 들어있는 Series 또는 배열.
    target_amount: 목표 금액.
    tolerance: float 비교를 위한 허용 오차.
    반환값: (True, [인덱스 리스트]) 또는 (False, [])
    """
    # DFS를 위해 candidates를 (인덱스, 금액) 튜플 리스트로 변환
    candidate_items = [(idx, amt) for idx, amt in candidates.items()]
    result_container = {'found': False, 'indices': []}

    def dfs(start_idx, current_sum, chosen_indices):
        if result_container['found']:
            return
        if np.abs(current_sum - target_amount) < tolerance:
            result_container['found'] = True
            result_container['indices'] = chosen_indices[:]
            return
        if start_idx >= len(candidate_items):
            return
        if current_sum > target_amount:
            return

        # 현재 원소 포함
        cand_idx, cand_amt = candidate_items[start_idx]
        dfs(start_idx + 1, current_sum + cand_amt, chosen_indices + [cand_idx])
        # 현재 원소 미포함
        dfs(start_idx + 1, current_sum, chosen_indices)

    dfs(0, 0.0, [])
    return (True, result_container['indices']) if result_container['found'] else (False, [])


def match_tax_and_book(df_tax, filtered_df_book, tolerance=1e-6):
    """
    df_tax (df_tax_new)와 filtered_df_book을 대사(매칭)하는 함수.

    1) df_tax의 각 행(대사 금액 = 공급가액 + 세액)에 대해
       - 1:1 매칭(매입금액대사) 시도
         → 매칭 성공 시, df_tax의 '구분키2', '차변금액', '전표번호', '회계일', '대사여부' 컬럼을 업데이트하고,
            filtered_df_book의 해당 행('Key', '구분키')를 갱신 (대사여부는 df_tax의 '대사여부' 값을 사용)
       - 1:1 매칭 실패 시, 후보들의 부분조합(매입순차대사(조합))을 탐색하여 매칭.
         → 조합 매칭 시, 후보들의 회계일 월이 모두 동일하지 않으면 df_tax의 '비고'에 "확인요청"을 표시하고,
            분할납부 내역(분할납부X_금액, 분할납부X_월)을 기록.
         → 매칭 성공 시, 각 후보에 대해 filtered_df_book의 'Key'와 '구분키'를 업데이트하고,
            df_tax의 '대사여부' 컬럼에는 기존 대사여부 값에 순번(예: "대사여부-1/2/3")을 부여.
    2) 최종적으로 업데이트된 df_tax와 filtered_df_book을 반환.

    ※ df_tax는 대사 대상 데이터프레임(df_tax_new)로, 공급가액, 세액, 업체사업자번호, 작성년도, 작성월, 국세청승인번호 등이 존재한다고 가정합니다.
       별도의 키 컬럼 대신 df_tax의 '대사여부' 컬럼 값을 대사 키로 사용합니다.
    """
    # 원본 보존을 위해 복사
    df_tax = df_tax.copy()
    filtered_df_book = filtered_df_book.copy()

    # df_tax 에 필요한 컬럼 생성
    if '구분키2' not in df_tax.columns:
        df_tax['구분키2'] = None
    if '차변금액' not in df_tax.columns:
        df_tax['차변금액'] = None
    if '전표번호' not in df_tax.columns:
        df_tax['전표번호'] = None
    if '회계일' not in df_tax.columns:
        df_tax['회계일'] = None
    if '비고' not in df_tax.columns:
        df_tax['비고'] = ""

    # filtered_df_book에 필요한 컬럼 생성
    if '구분키' not in filtered_df_book.columns:
        filtered_df_book['구분키'] = ""
    if 'Key' not in filtered_df_book.columns:
        filtered_df_book['Key'] = None

    # filtered_df_book의 '회계일' 컬럼을 datetime 형으로 변환 (필요 시)
    if not pd.api.types.is_datetime64_any_dtype(filtered_df_book['회계일']):
        filtered_df_book['회계일'] = pd.to_datetime(filtered_df_book['회계일'], errors='coerce')

    # df_tax의 각 행에 대해 매칭 처리
    for idx, tax_row in df_tax.iterrows():
        # 이미 대사처리된 경우 건너뜁니다.
        # (여기서는 '구분키' 또는 '구분키2'가 채워져 있으면 이미 매칭된 것으로 간주)
        if tax_row['구분키'] in [None, "", np.nan]:
            continue
        if tax_row['구분키2'] not in [None, "", np.nan]:
            continue

        # 대사금액 계산: 공급가액 + 세액
        pivot_amount = tax_row['공급가액'] + tax_row['세액']
        pivot_biz = tax_row['업체사업자번호']
        pivot_year = tax_row['작성년도']
        pivot_month = tax_row['작성월']

        # 허용 회계일 범위: pivot_month의 1일부터 pivot_month+2개월의 마지막 날까지
        allowed_lower = pd.Timestamp(pivot_year, pivot_month, 1)
        allowed_upper = allowed_lower + pd.DateOffset(months=2) - pd.DateOffset(days=1)

        # filtered_df_book에서 후보 필터링: 거래처번호 일치, '구분키'가 빈 문자열, 회계일이 허용 범위 내
        candidates = filtered_df_book[
            (filtered_df_book['거래처번호'] == pivot_biz) &
            (filtered_df_book['구분키'] == "") &
            (filtered_df_book['회계일'] >= allowed_lower) &
            (filtered_df_book['회계일'] <= allowed_upper)
        ]
        if candidates.empty:
            continue

        # 회계일 기준 오름차순 정렬
        candidates = candidates.sort_values(by='회계일')

        # 1) 1:1 매칭 시도: 후보의 차변금액과 pivot_amount의 차이가 tolerance 이하이면
        exact_match = candidates[np.abs(candidates['차변금액'] - pivot_amount) < tolerance]
        if not exact_match.empty:
            candidate_index = exact_match.index[0]
            candidate_row = candidates.loc[candidate_index]

            df_tax.at[idx, '구분키2'] = "매입금액대사"
            df_tax.at[idx, '차변금액'] = candidate_row['차변금액']
            df_tax.at[idx, '전표번호'] = candidate_row['전표번호']
            df_tax.at[idx, '회계일'] = candidate_row['회계일'].strftime("%Y-%m-%d")

            # 1:1 매칭 시에는 기존의 대사여부 값을 그대로 사용 (예: "국세청승인번호-1" 등)
            # filtered_df_book 업데이트 (대사 키로 df_tax의 '대사여부' 사용)
            filtered_df_book.at[candidate_index, 'Key'] = tax_row['대사여부']
            filtered_df_book.at[candidate_index, '구분키'] = "매입금액대사"
            continue

        # 2) 1:1 매칭 실패 시, 부분조합(매입순차대사(조합)) 매칭 시도 (순차대사 로직)
        subset_found, subset_indices = find_subset_sum_all_combinations(
            candidates['차변금액'],
            pivot_amount,
            tolerance
        )
        if subset_found and len(subset_indices) > 0:
            subset_cands = candidates.loc[subset_indices]
            df_tax.at[idx, '구분키2'] = "매입순차대사(조합)"
            df_tax.at[idx, '차변금액'] = subset_cands['차변금액'].sum()
            # 여러 건 중 첫번째 후보의 전표번호 사용
            df_tax.at[idx, '전표번호'] = subset_cands.iloc[0]['전표번호']
            df_tax.at[idx, '회계일'] = subset_cands['회계일'].max().strftime("%Y-%m-%d")

            # 후보 조합의 회계일 월이 모두 동일하지 않으면 "확인요청" 및 분할납부 내역 기록
            unique_months = subset_cands['회계일'].dt.month.unique()
            if len(unique_months) > 1:
                df_tax.at[idx, '비고'] = "확인요청"
                subset_cands = subset_cands.copy()
                subset_cands['회계월'] = subset_cands['회계일'].dt.strftime('%Y-%m')
                monthly_group = subset_cands.groupby('회계월', as_index=False)['차변금액'].sum()
                for j, row in monthly_group.iterrows():
                    amount_col = f"분할납부{j+1}_금액"
                    month_col = f"분할납부{j+1}_월"
                    df_tax.at[idx, amount_col] = row['차변금액']
                    df_tax.at[idx, month_col] = row['회계월']

            # 각 후보에 대해 filtered_df_book 업데이트 (순번 부여)
            for i, si in enumerate(subset_indices, start=1):
                filtered_df_book.at[si, 'Key'] = tax_row['대사여부']
                filtered_df_book.at[si, '구분키'] = f"매입순차대사(조합)-{i}"
            seq_numbers = "/".join(str(i) for i in range(1, len(subset_indices)+1))
    return df_tax, filtered_df_book

# ========== 사용 예시 ==========
# df_tax_new (대사 대상 데이터프레임)와 filtered_df_book 데이터프레임이 미리 준비되어 있다고 가정합니다.
df_tax_new, filtered_df_book = match_tax_and_book(df_tax_new, filtered_df_book)

print(df_tax_new.head())
print(filtered_df_book.head())

    협력사코드      계산서작성일             협력사명    계산서구분         사업자번호        공급가액  \
0  429710  2024-01-31  (식)세림비앤지-신(OMS)  일반세금계산서  262-87-01457    156000.0   
1  427211  2024-01-31       (식)지원(OMS)  일반세금계산서  128-81-24750   1398915.0   
2  427211  2024-01-31       (식)지원(OMS)  일반세금계산서  128-81-24750    121437.0   
3  429710  2024-01-31  (식)세림비앤지-신(OMS)  일반세금계산서  262-87-01457   5399280.0   
4  428262  2024-01-31   (식경)한국이노팩(OMS)  일반세금계산서  387-81-00191  57037360.0   

          세액                   국세청승인번호     업체사업자번호     국세청작성일     국세청발급일  \
0    15600.0  2024013142000076y0258221  2628701457 2024-01-31 2024-02-06   
1   139892.0  2024013142000076y0258306  1288124750 2024-01-31 2024-02-06   
2    12144.0  2024013142000076y0258307  1288124750 2024-01-31 2024-02-06   
3   539928.0  2024013142000076y0255900  2628701457 2024-01-31 2024-02-05   
4  5703736.0  2024013142000076y0256132  3878100191 2024-01-31 2024-02-05   

   작성년도  작성월              대사여부   구분키    구분키2        차변금액           전표번호  \
0  20

In [58]:
df_tax_new.to_excel("세금계산서파일확인.xlsx")

In [59]:
# 업체사업자번호, 협력사코드, 년월 순서로 오름차순 정렬
df_final_pivot = df_final_pivot.sort_values(by=["업체사업자번호", "협력사코드", "년월"], ascending=True)
filtered_df_book = filtered_df_book.sort_values(by=['거래처번호', '회계일'], ascending=True)

In [60]:
df_tax_new['회계일'] = pd.to_datetime(df_tax_new['회계일'], errors='coerce')
df_tax_new['국세청발급일'] = pd.to_datetime(df_tax_new['국세청발급일'], errors='coerce')

# Excel 수식 =DAYS(회계일, 국세청발급일)와 동일하게 일수 차이를 계산하여 '기간' 컬럼에 저장합니다.
df_tax_new['기간'] = (df_tax_new['회계일'] - df_tax_new['국세청발급일']).dt.days

In [61]:
conditions = [
    (df_tax_new['기간'] <= 10),
    (df_tax_new['기간'] > 10) & (df_tax_new['기간'] <= 15),
    (df_tax_new['기간'] > 15) & (df_tax_new['기간'] <= 30),
    (df_tax_new['기간'] > 30)
]

# 각 조건에 해당하는 값 설정
choices = [1, 2, 3, 4]

# np.select를 이용하여 '일수구분' 컬럼 생성 (해당 조건에 해당하는 값 할당)
df_tax_new['일수구분'] = np.select(conditions, choices, default=np.nan)

In [62]:
# 시작일, 종료일 지정 (비교를 위해 full date로 변환)
start_date = pd.to_datetime("2024-01-01")
end_date = pd.to_datetime("2024-06-30")

# df_tax_new의 회계일, 국세청발급일 컬럼이 datetime 타입인지 확인 (필요 시 변환)
df_tax_new['회계일'] = pd.to_datetime(df_tax_new['회계일'], errors='coerce')
df_tax_new['국세청발급일'] = pd.to_datetime(df_tax_new['국세청발급일'], errors='coerce')

# 대사여부가 존재하는 경우
mask_exist = df_tax_new['대사여부'].notnull() & (df_tax_new['대사여부'] != "")

# (1) 구분키2가 있는 경우
mask_with = mask_exist & ~(df_tax_new['구분키2'].isnull() | (df_tax_new['구분키2'] == ""))
# 그 중 회계일이 start_date 이전인 경우 --> "이전기간 지급"
mask_prev = mask_with & (df_tax_new['회계일'] < start_date)

# (2) 구분키2가 없는 경우
mask_without = mask_exist & ((df_tax_new['구분키2'].isnull()) | (df_tax_new['구분키2'] == ""))
# 그 중 국세청발급일이 end_date 이후이면 --> "이후기간 지급"
mask_after = mask_without & (df_tax_new['국세청발급일'] > end_date)
# 나머지 --> "확인필요"
mask_remaining = mask_without & ~(mask_after)

# 조건에 따라 비고 컬럼 값 업데이트
df_tax_new.loc[mask_prev, '비고'] = "이전기간 지급"
df_tax_new.loc[mask_after, '비고'] = "이후기간 지급"
df_tax_new.loc[mask_remaining, '비고'] = "확인필요"

In [63]:
# 1. df_tax_new의 대사여부에서 기본 key 추출 (예: "A123-1"인 경우 "A123"을 추출)
df_tax_new['base_key'] = df_tax_new['대사여부'].astype(str).str.split('-').str[0]

# 2. 특별 집계에 해당하는 비고 값 설정 (이전기간 지급, 이후기간 지급, 확인필요)
special_flags = ["이전기간 지급", "이후기간 지급", "확인필요"]

# 3. 일반 집계: 비고 컬럼이 special_flags에 포함되지 않은 행만 선택
normal_df = df_tax_new[~df_tax_new['비고'].isin(special_flags)]

# 4. 일반 집계: base_key와 일수구분 기준으로 그룹화하여 차변금액 합산
agg_df = normal_df.groupby(['base_key', '일수구분'])['차변금액'].sum().reset_index()

# 5. 피벗테이블 형식으로 변경 (일수구분별 합계를 한 행에 표시)
pivot_agg_normal = agg_df.pivot(index='base_key', columns='일수구분', values='차변금액').reset_index()

# 6. 일반 집계 피벗에서 일수구분 컬럼명을 원하는 명칭으로 변경
pivot_agg_normal = pivot_agg_normal.rename(columns={
    1: '합계_10일이하',
    2: '합계_10초과_15일이하',
    3: '합계_15초과_30일이하',
    4: '합계_30일초과'
})

# 7. 특별 집계: "이전기간 지급"은 그대로 차변금액의 합계로 산출
agg_prev = (df_tax_new[df_tax_new['비고'] == "이전기간 지급"]
            .groupby('base_key')['차변금액']
            .sum()
            .reset_index()
            .rename(columns={'차변금액': '이전기간 지급'}))

# 8. 특별 집계: "이후기간 지급"은 공급가액과 세액의 합계 기준으로 산출
df_temp_after = df_tax_new[df_tax_new['비고'] == "이후기간 지급"].copy()
df_temp_after['공급세액합계'] = df_temp_after['공급가액'] + df_temp_after['세액']
agg_after = (df_temp_after.groupby('base_key')['공급세액합계']
             .sum()
             .reset_index()
             .rename(columns={'공급세액합계': '이후기간 지급'}))

# 9. 특별 집계: "확인필요"는 공급가액과 세액의 합계 기준으로 산출
df_temp_need = df_tax_new[df_tax_new['비고'] == "확인필요"].copy()
df_temp_need['공급세액합계'] = df_temp_need['공급가액'] + df_temp_need['세액']
agg_need = (df_temp_need.groupby('base_key')['공급세액합계']
            .sum()
            .reset_index()
            .rename(columns={'공급세액합계': '확인필요'}))

# 10. 일반 집계 피벗과 각 특별 집계 결과를 base_key를 기준으로 병합
pivot_agg = pd.merge(pivot_agg_normal, agg_prev, on='base_key', how='outer')
pivot_agg = pd.merge(pivot_agg, agg_after, on='base_key', how='outer')
pivot_agg = pd.merge(pivot_agg, agg_need, on='base_key', how='outer')

# 11. df_final_pivot과 pivot_agg를 df_final_pivot의 key와 pivot_agg의 base_key 기준으로 병합 (1:n 관계)
merged_df = pd.merge(df_final_pivot, pivot_agg, left_on='key', right_on='base_key', how='left')
merged_df.drop('base_key', axis=1, inplace=True)

In [64]:
# 만약 '최종매입금액' 컬럼이 없다면, 예시로 '차변금액' 컬럼을 사용하여 생성 (필요에 맞게 수정하세요)
if '최종지불금액' not in merged_df.columns:
    merged_df['최종지불금액'] = merged_df['최종매입금액']

# 기본적으로 최종매입금액을 최종지불금액에 할당
merged_df['최종지불금액'] = merged_df['최종매입금액']

# 과세: 과세구분이 '과세'인 경우 → 최종매입금액에 1.1을 곱함
mask_taxable = merged_df['면과세구분명'] == '과세'
merged_df.loc[mask_taxable, '최종지불금액'] = merged_df.loc[mask_taxable, '최종매입금액'] * 1.1




In [65]:
import numpy as np
import pandas as pd

# 할당에 사용할 피벗 컬럼들이 존재하지 않으면 기본값 0.0을 추가합니다.
for col in ['합계_10일이하', '합계_10초과_15일이하', '합계_15초과_30일이하']:
    if col not in merged_df.columns:
        merged_df[col] = 0.0

def allocate_update(row):
    # 구분키가 '부분대사' 또는 '수기확인'인 경우에만 처리
    if row.get('구분키') in ['부분대사', '수기확인']:
        # 피벗 컬럼의 원래 값을 먼저 저장 (할당 전의 값을 보존)
        orig_10 = row.get('합계_10일이하', 0.0)
        orig_10 = 0.0 if pd.isnull(orig_10) else orig_10

        orig_1015 = row.get('합계_10초과_15일이하', 0.0)
        orig_1015 = 0.0 if pd.isnull(orig_1015) else orig_1015

        orig_1530 = row.get('합계_15초과_30일이하', 0.0)
        orig_1530 = 0.0 if pd.isnull(orig_1530) else orig_1530

        # 특별 지급금액(이전기간 지급, 이후기간 지급, 확인필요)을 안전하게 가져옵니다.
        prev_payment = row.get('이전기간 지급', 0.0)
        prev_payment = 0.0 if pd.isnull(prev_payment) else prev_payment

        after_payment = row.get('이후기간 지급', 0.0)
        after_payment = 0.0 if pd.isnull(after_payment) else after_payment

        need_payment = row.get('확인필요', 0.0)
        need_payment = 0.0 if pd.isnull(need_payment) else need_payment

        special_sum = prev_payment + after_payment + need_payment

        # 최종지불금액에서 특별 지급금액을 먼저 차감합니다.
        target = row['최종지불금액']
        new_target = target - special_sum

        if new_target <= 0:
            alloc_1530 = alloc_1015 = alloc_10 = 0.0
        else:
            remain = new_target

            # 먼저 '합계_15초과_30일이하'에서 할당
            avail_1530 = orig_1530
            alloc_1530 = min(avail_1530, remain)
            remain -= alloc_1530

            # 다음 '합계_10초과_15일이하'에서 할당
            avail_1015 = orig_1015
            alloc_1015 = min(avail_1015, remain)
            remain -= alloc_1015

            # 마지막으로 '합계_10일이하'에서 할당
            avail_10 = orig_10
            alloc_10 = min(avail_10, remain)
            remain -= alloc_10

        # 피벗 컬럼을 새로 할당된 값으로 업데이트합니다.
        row['합계_15초과_30일이하'] = alloc_1530
        row['합계_10초과_15일이하'] = alloc_1015
        row['합계_10일이하'] = alloc_10

        # 차이내역: 원래 피벗 컬럼의 합계에서 새로 할당된 금액의 합계를 차감한 값
        total_orig = orig_10 + orig_1015 + orig_1530
        total_alloc = alloc_10 + alloc_1015 + alloc_1530
        row['차이내역'] = total_orig - total_alloc

    return row

# 각 행마다 할당 처리 후 merged_df에 업데이트 적용
final_merged_df = merged_df.apply(allocate_update, axis=1)

# 결과 확인
print(final_merged_df[['구분키', '최종지불금액', '이전기간 지급', '이후기간 지급', '확인필요',
                 '합계_10일이하', '합계_10초과_15일이하', '합계_15초과_30일이하', '차이내역']].head())


    구분키      최종지불금액  이전기간 지급  이후기간 지급  확인필요  합계_10일이하  합계_10초과_15일이하  \
0  금액대사   8023092.0      NaN      NaN   NaN       NaN      8023092.0   
1  부분대사   8719920.0      NaN      NaN   NaN       0.0      8719920.0   
2  부분대사  11469480.0      NaN      NaN   NaN       0.0     11469480.0   
3  부분대사  10550760.0      NaN      NaN   NaN       0.0            0.0   
4  부분대사   8951580.0      NaN      NaN   NaN       0.0            0.0   

   합계_15초과_30일이하        차이내역  
0            NaN         NaN  
1            0.0  34315829.0  
2            0.0  19922364.0  
3     10550760.0  19703904.0  
4      8951580.0   2894430.0  


In [66]:
final_merged_df = final_merged_df[["협력사코드", "협력사명", '년월', '면과세구분명', '최종매입금액', '구분키', 'key', '업체사업자번호','최종지불금액', '합계_10일이하', '합계_10초과_15일이하', '합계_15초과_30일이하','차이내역','이전기간 지급', '이후기간 지급','확인필요', '지불예상금액']]

In [67]:
sheets_data = [
    ("최종총괄장내역", final_merged_df),
    ("대사총괄장내역", merged_df),
    ("지불보조장내역", filtered_df_book),
    ("세금계산서내역", df_tax_new),
    ("요청단품내역", df_standard)
]

def df_to_sheet(sheet, df, start_row=1, start_col=1):
    """
    DataFrame의 내용을 Excel Sheet에 기록하는 함수입니다.
    헤더부터 기록하며, (start_row, start_col) 위치부터 작성합니다.
    Timestamp인 경우 tz-naive이면 UTC로 localize 후 Python datetime으로 변환하며,
    pd.NaT (결측치)는 빈 문자열로 대체합니다.
    """
    n_rows, n_cols = df.shape

    # 헤더 작성
    for col_idx, col_name in enumerate(df.columns.values, start=start_col):
        sheet.Cells(start_row, col_idx).Value = col_name

    # 데이터 작성 (헤더 바로 아래 행부터 기록)
    for row_idx in range(n_rows):
        for col_idx in range(n_cols):
            val = df.iat[row_idx, col_idx]
            # 결측치(NaT, None 등)는 빈 문자열로 처리
            if pd.isnull(val):
                val = ""
            elif isinstance(val, pd.Timestamp):
                if val.tzinfo is None:
                    # tz-naive인 경우 UTC로 localize 후 Python datetime 객체로 변환
                    val = val.tz_localize("UTC").to_pydatetime()
                else:
                    val = val.to_pydatetime()
            sheet.Cells(start_row + row_idx + 1, col_idx + start_col).Value = val

def apply_styles(sheet, last_row, last_col):
    """
    지정한 범위에 대해 얇은 테두리, 정렬, 숫자 서식, 헤더 배경색 및 열 너비 자동 조정을 적용하는 함수입니다.
    """
    used_range = sheet.Range(sheet.Cells(1, 1), sheet.Cells(last_row, last_col))
    used_range.Borders.LineStyle = 1  # 얇은 테두리 적용

    # 헤더 스타일: 굵은 글씨, 중앙 정렬, 회색 배경
    header_range = sheet.Range(sheet.Cells(1, 1), sheet.Cells(1, last_col))
    header_range.Font.Bold = True
    header_range.HorizontalAlignment = -4108  # xlCenter
    header_range.VerticalAlignment = -4108
    header_range.Interior.Color = 0xD3D3D3  # 회색 (RGB 211,211,211)

    # 데이터 영역: 오른쪽 정렬
    data_range = sheet.Range(sheet.Cells(2, 1), sheet.Cells(last_row, last_col))
    data_range.HorizontalAlignment = -4152  # xlRight

    # 열 너비 자동 조정
    sheet.Columns.AutoFit()

def save_excel_with_pywin(save_path):
    """
    pywin (win32com)을 사용하여 Excel 파일을 생성하고,
    각 시트에 sheets_data 순서대로 데이터를 기록 및 스타일 적용 후 저장하는 함수입니다.
    Excel 창은 보이지 않게 실행됩니다.
    """
    # 저장 경로가 절대 경로로 보이도록 변환
    save_path = os.path.abspath(save_path)
    # 저장 경로의 폴더가 존재하지 않으면 생성합니다.
    os.makedirs(os.path.dirname(save_path), exist_ok=True)

    # Excel 애플리케이션 실행 (보이지 않게)
    excel = win32.Dispatch("Excel.Application")
    excel.Visible = False        # Excel 창을 보이지 않게 설정
    excel.DisplayAlerts = False  # 알림창 표시 안함

    # 새 워크북 생성 (기본 시트 1개가 있음)
    wb = excel.Workbooks.Add()

    # sheets_data의 첫 번째 DataFrame을 기본 시트에 기록
    first_sheet_name, first_df = sheets_data[0]
    ws = wb.Worksheets(1)
    ws.Name = first_sheet_name
    n_rows, n_cols = first_df.shape
    df_to_sheet(ws, first_df, start_row=1, start_col=1)
    apply_styles(ws, last_row=n_rows + 1, last_col=n_cols)

    # sheets_data의 나머지 시트를 순서대로 추가하면서 기록
    for sheet_name, df in sheets_data[1:]:
        ws = wb.Worksheets.Add(After=wb.Worksheets(wb.Worksheets.Count))
        ws.Name = sheet_name
        n_rows, n_cols = df.shape
        df_to_sheet(ws, df, start_row=1, start_col=1)
        apply_styles(ws, last_row=n_rows + 1, last_col=n_cols)

    # XLSX 형식(FileFormat=51)으로 저장
    wb.SaveAs(save_path, FileFormat=51)
    wb.Close(False)
    excel.Quit()
    display(f"Excel 파일이 저장되었습니다: {save_path}")

In [ ]:
# 현재 날짜를 YYYYMMDD 형식 문자열로 구함
today = datetime.datetime.now().strftime("%Y%m%d")
# 파일 이름에 현재 날짜를 붙임 (예: '최종파일_20231007.xlsx')
filename = f"최종파일_{today}.xlsx"
save_path = os.path.join("output", filename)

# 파일을 저장
save_excel_with_pywin(save_path)

'Excel 파일이 저장되었습니다: c:\\Users\\Administrator\\Desktop\\subcontract\\output\\최종파일_20250708.xlsx'

: 